In [19]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import Normalization
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Flatten
from keras import layers

In [88]:
df_merged = pd.read_csv("df_merged.csv")

In [89]:
df_merged["created_at"] = pd.to_datetime(df_merged["created_at"])

In [90]:
df_merged = df_merged.groupby([pd.Grouper(key='created_at',freq='D'), "party"]).agg({
    "replies_count": "sum", 
    "retweets_count": "sum",
    "likes_count": "sum",
    "avg_len_of_tweet": "mean" ,
    "share_of_tweets" :"mean",
    "share_unique_users": "mean",
    "weighted_sentiment" : "mean",
    "share_of_positive_tweets": "mean",
    "share_of_negative_tweets" : "mean",
    "Polls": "mean"}) 

In [94]:
df_merged.head(14)

replies_count  retweets_count  likes_count  \
created_at party                                                
2021-05-26 AFD               0.0             0.0          0.0   
           CDU               0.0             0.0          0.0   
           FDP               0.0             0.0          0.0   
           GRUENE            0.0             0.0          0.0   
           LINKE             0.0             0.0          0.0   
           SPD               0.0             0.0          0.0   
           other             0.0             0.0          0.0   
2021-05-27 AFD               0.0             0.0          0.0   
           CDU               0.0             0.0          0.0   
           FDP               0.0             0.0          0.0   
           GRUENE            0.0             0.0          0.0   
           LINKE             0.0             0.0          0.0   
           SPD               0.0             0.0          0.0   
           other             0.0             0.0          0.0   

                   avg_len_of_tweet  share_of_tweets  share_unique_users  \
created_at party                                                           
2021-05-26 AFD                  NaN              NaN                 NaN   
           CDU                  NaN              NaN                 NaN   
           FDP                  NaN              NaN                 NaN   
           GRUENE               NaN              NaN                 NaN   
           LINKE                NaN              NaN                 NaN   
           SPD                  NaN              NaN                 NaN   
           other                NaN              NaN                 NaN   
2021-05-27 AFD                  NaN              NaN                 NaN   
           CDU                  NaN              NaN                 NaN   
           FDP                  NaN              NaN                 NaN   
           GRUENE               NaN              NaN                 NaN   
           LINKE                NaN              NaN                 NaN   
           SPD                  NaN              NaN                 NaN   
           other                NaN              NaN                 NaN   

                   weighted_sentiment  share_of_positive_tweets  \
created_at party                                                  
2021-05-26 AFD                    NaN                       NaN   
           CDU                    NaN                       NaN   
           FDP                    NaN                       NaN   
           GRUENE                 NaN                       NaN   
           LINKE                  NaN                       NaN   
           SPD                    NaN                       NaN   
           other                  NaN                       NaN   
2021-05-27 AFD                    NaN                       NaN   
           CDU                    NaN                       NaN   
           FDP                    NaN                       NaN   
           GRUENE                 NaN                       NaN   
           LINKE                  NaN                       NaN   
           SPD                    NaN                       NaN   
           other                  NaN                       NaN   

                   share_of_negative_tweets  Polls  
created_at party                                    
2021-05-26 AFD                          NaN  14.38  
           CDU                          NaN  25.50  
           FDP                          NaN  11.62  
           GRUENE                       NaN  20.25  
           LINKE                        NaN   7.88  
           SPD                          NaN  14.00  
           other                        NaN   6.38  
2021-05-27 AFD                          NaN  17.50  
           CDU                          NaN  27.00  
           FDP                          NaN  10.00  
           GRUENE                       NaN  15.50  

In [92]:
model = Sequential()
model.add(layers.LSTM(units=20, activation='tanh', return_sequences=True))
model.add(layers.LSTM(units=20, activation='tanh', return_sequences=False))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(7, activation='softmax'))


In [95]:
X= df_merged
y = df_merged['Polls']

In [96]:
y.shape

(634,)

In [97]:
X.shape

(634, 10)

In [98]:


model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.fit(X, y ,
              epochs=1, 
              batch_size=1, 
              verbose=1, 
              
              validation_split=0.2)



ValueError: in user code:

    /home/lucab/.pyenv/versions/3.8.6/envs/project_delphi/lib/python3.8/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /home/lucab/.pyenv/versions/3.8.6/envs/project_delphi/lib/python3.8/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/lucab/.pyenv/versions/3.8.6/envs/project_delphi/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/lucab/.pyenv/versions/3.8.6/envs/project_delphi/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/lucab/.pyenv/versions/3.8.6/envs/project_delphi/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_f

# MAKE DATA STATIONARY?//???